<a href="https://colab.research.google.com/github/sayanbanerjee32/NLP-with-fastai2.0/blob/main/fastai_on_10kGNAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#!pip install fastai==2.1.4
#!pip install bpemb
#!pip install cleantext 

In [31]:
# !wget https://github.com/tblock/10kGNAD/raw/master/train.csv
# !wget https://github.com/tblock/10kGNAD/raw/master/test.csv
# !ls

--2020-11-17 09:56:53--  https://github.com/tblock/10kGNAD/raw/master/train.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv [following]
--2020-11-17 09:56:54--  https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24405789 (23M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  23.27M  56.1MB/s    in 0.4s    

2020-11-17 09:56:54 (56.1 MB/s) - ‘train.csv’ saved [24405789/24405789]

--2020-11-17 09:56:54--  https://github.com/tblock/10kGNAD/raw/master/test.csv
Resolving g

In [1]:
!mkdir nltk_data
!ls

10kGNAD  nltk_data  sample_data  test.csv  train.csv


In [10]:
import nltk
nltk.download('stopwords', download_dir='nltk_data')
nltk.data.path.append('nltk_data')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
import csv

from bpemb import BPEmb
from cleantext import clean
#from fastai.callbacks import *
#from fastai.imports import torch
from fastai.text import * 
import pandas as pd

#torch.cuda.set_device(2)

In [4]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

# construct the vocabulary by added a padding token with the ID 25000 (because of the bpemb_de vocab size)
itos = dict(enumerate(bpemb_de.words + ['xxpad']))
#voc = Vocab(itos)

In [14]:
def load_data(filename):
    texts = []
    labels = []
    with open(filename) as csvfile:
        # follow the 10kGNAD creator's setup
        reader = csv.reader(csvfile, delimiter=';', quotechar='\'')
        for row in reader:
            labels.append(row[0])
            texts.append(row[1])
    df = pd.DataFrame({'label': labels, 'text': texts})
    df['text'] = df['text'].apply(lambda x: bpemb_de.encode_ids_with_bos_eos(clean(x, stp_lang='german')))
                                                                                   #lang='de')))
    return df

df_train_valid = load_data("train.csv")

# the last 1000 training samples are used for validation
df_train = df_train_valid.iloc[:-1000]
df_valid = df_train_valid.iloc[-1000:]

df_test = load_data("test.csv")

In [15]:
df_train.head()

,label,text
0,Sport,"[1, 2271, 7548, 6787, 2565, 206, 13453, 123, 2..."
1,Kultur,"[1, 16382, 24915, 4984, 114, 8386, 24940, 31, ..."
2,Web,"[1, 26, 12053, 77, 15950, 13, 125, 8827, 14713..."
3,Wirtschaft,"[1, 14253, 7, 24948, 1195, 23, 1021, 24940, 36..."
4,Inland,"[1, 18392, 5417, 91, 4575, 9211, 981, 122, 559..."
